<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_overlap0_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_0.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,3,cracking sensations in my joints
2,4,purple sploches around waist
3,5,fast and slow heartbeat
4,5,increased heart rate


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'is',
 'my',
 'myalgia',
 'lower',
 'rt',
 'fatigue',
 'loss',
 'by',
 'cramp',
 'limb',
 'weakness',
 'sodium',
 'at',
 'disease',
 'that',
 'stomach',
 'or',
 'back',
 'as',
 '-',
 'depression',
 'patients',
 'from',
 'i',
 'this',
 'cramps',
 'not',
 'joint',
 'upper',
 'gas',
 'drug',
 'leg',
 'memory',
 'mg',
 'you',
 'all',
 'pains',
 'legs',
 'headache',
 'group',
 'be',
 'knee',
 'shoulder',
 'are',
 'unable',
 'study',
 'arthralgia',
 'treatment',
 'gastrointestinal',
 'neck',
 'excessive',
 'after',
 'arthritis',
 'aches',
 'an',
 'foot',
 'it',
 'hip',
 'amp',
 'have',
 'voltaren',
 'like',
 'numbness',
 'arthrotec',
 'insomnia',
 'increased',
 'p',
 'no',
 'both',
 '/',
 'release',
 'bleeding',
 'nausea',
 'gluten',
 'can',
 'time',
 'asthenia',

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.844141,3.491211,0.440647,35:21


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.519639,3.432925,0.446326,35:36


In [25]:
learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_bigtweet_askpatient_mixed_overlap0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.940258,3.500388,0.382249,01:16


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.994701,3.085384,0.453254,01:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.246021,2.622950,0.576331,03:13


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.980936,2.265943,0.617751,03:35
1,1.708008,2.133132,0.644970,03:37


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.488198,1.911959,0.666272,03:22
1,1.281131,1.799770,0.682840,03:22


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.281264,1.758745,0.681657,03:24
1,1.074082,1.641008,0.708876,03:33


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.950677,1.579004,0.720710,03:25
1,0.884919,1.493146,0.739645,03:29


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.810173,1.484764,0.742012,03:36
1,0.690307,1.430711,0.755030,03:18


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.665745,1.388863,0.757396,03:36
1,0.667520,1.369364,0.760947,03:15
2,0.562180,1.297689,0.781065,03:43
3,0.481150,1.291107,0.782248,03:15


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.505376,1.339105,0.783432,03:15
1,0.506888,1.320771,0.772781,03:38
2,0.470805,1.250291,0.784615,03:11
3,0.359843,1.252890,0.789349,03:22


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.381698,1.271582,0.792899,03:12
1,0.405435,1.271992,0.792899,03:34
2,0.362969,1.261559,0.796450,03:19
3,0.293535,1.235469,0.800000,03:43


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.284610,1.277482,0.795266,03:25
1,0.350604,1.328014,0.801183,03:22
2,0.291348,1.248400,0.803550,03:26
3,0.271815,1.251564,0.804734,03:23


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.272724,1.259453,0.794083,03:28
1,0.313890,1.351903,0.803550,03:24
2,0.248327,1.327967,0.809467,03:32
3,0.218422,1.246281,0.805917,03:26


In [39]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [40]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [41]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [42]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.223610,1.385077,0.795266,03:33


In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.252709,1.325123,0.796450,03:12


In [49]:
# save the best model

learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap0')

# Part three: Predict on the test dataset

In [50]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [51]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

610
610
[90, 126, 15, 334, 5, 933, 218, 760, 10, 685, 5, 808, 555, 27, 644, 595, 29, 76, 211, 30, 34, 289, 38, 623, 41, 35, 218, 218, 325, 475, 685, 623, 70, 71, 71, 76, 265, 717, 1026, 79, 146, 293, 289, 863, 80, 15, 89, 391, 90, 850, 94, 810, 337, 98, 708, 98, 757, 337, 100, 337, 100, 591, 100, 291, 236, 107, 108, 108, 209, 315, 313, 111, 511, 111, 111, 111, 293, 730, 937, 831, 218, 808, 125, 546, 187, 579, 129, 129, 289, 745, 607, 144, 591, 218, 430, 144, 633, 509, 178, 166, 405, 170, 607, 180, 127, 182, 182, 182, 182, 973, 189, 190, 190, 190, 190, 190, 190, 190, 197, 34, 34, 34, 34, 736, 764, 769, 210, 210, 210, 210, 159, 211, 896, 411, 471, 214, 348, 214, 1026, 973, 587, 44, 228, 79, 228, 228, 228, 544, 231, 234, 236, 337, 402, 973, 853, 772, 328, 392, 156, 973, 973, 973, 156, 765, 245, 245, 245, 245, 245, 245, 467, 246, 644, 516, 973, 618, 259, 289, 260, 156, 265, 419, 660, 402, 265, 704, 265, 265, 199, 86, 391, 504, 485, 281, 807, 283, 355, 805, 419, 696, 289, 289, 289, 289, 861

In [52]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

250
0.4098360655737705
